# Vorlesung 9.2
# Bitfelder und Aufzählungen

---
 **Hinweis:**
 Diese interaktiven Webseiten beschreiben parallel zu den Vorlesungsfolien das jeweilige Stoffgebiet. Zellen mit C Quelltext können mittels der Tastenkombination \[Shift\] + \[Enter\] kompiliert und ausgeführt werden. Es wird Ihnen empfohlen, Änderungen in diversen Zellen vorzunehmen um ein Gefühl für die Sprache C zu entwickeln.
 
---

# Bitfelder

Der kleinste Datentyp ist `char` und benötigt 8 Bit Speicherplatz. Mit Bitfeldern können auch kleinere Objekte gespeichert werden. Die Definition ist ähnlich wie bei Strukturen:

In [ ]:
#include <stdio.h>

/* Definition eines Bitfeldes mit insgesamt 8 Bit, also 1 Byte.
    Das Bitfeld ist damit nur 1 Byte groß */
typedef struct Bitfeld_s {
    unsigned char a : 3; /* a belegt drei Bit im Speicher */
    unsigned char b : 2; /* b nur 2 Bit */
    unsigned char c : 3; /* c wieder 3 Bit */
} Bitfeld_t;

int main() {
    Bitfeld_t bf;
    /* Speichergröße von Bitfeld_t */
    printf("Speichergroesse: %ld Byte\n", sizeof(Bitfeld_t));
    
    /* Die Variablen können nun gesetzt werden, wie in einem struct. */
    bf.a = 0;
    bf.b = 0;
    bf.c = 0;
    
    /* Allerdings ist ihr Speicherplatz auf die angegebene Anzahl der Bit beschränkt.
        Wenn noch mehr addiert wird, beginnen sie wieder bei 0 */
    {
        long i = 0;
        printf("a  b  c\n");
        for(i = 0; i < 10; ++i) {
            printf("%d  %d  %d\n", bf.a, bf.b, bf.c);
            ++bf.a;
            ++bf.b;
            ++bf.c;
        }
    }
    
    return 0;
}

## Datentypen in Bitfeldern

In Bitfeldern können nur Ganzzahlen als Datentypen verwendet werden. Die Größe der gesamten Struktur ist ein vielfaches dieses Datentyps. Die genaue Speicheraufteilung ist aber nicht genau festgelegt und kann auf verschiedenen Betriebssystemen variieren. 

**Achtung**: Der Server, der im Hintergrund dieses Notebook ausführt, unterstützt auch `char` (1 Byte) als minimal Größe eines Bitfeldes. Das wird nicht von jedem Compiler unterstützt.

In [ ]:
#include <stdio.h>

/* Definition eines Bitfeldes mit insgesamt 8 Bit, also 1 Byte.
    Das Bitfeld ist aber nun 4 Byte groß weil unsigned int als Datentyp gewählt wurde */
typedef struct Bitfeld_s {
    unsigned int a : 3; /* a belegt drei Bit im Speicher */
    unsigned int b : 2; /* b nur 2 Bit */
    unsigned int c : 3; /* c wieder 3 Bit */
} Bitfeld_t;

int main() {
    /* Speichergröße von Bitfeld_t */
    printf("Speichergroesse: %ld Byte\n", sizeof(Bitfeld_t));
    
    return 0;
}

## Verwendung von Bitfeldern

### Kleine Datenmengen

Bitfelder werden oft verwendet um kleine Datenmengen effizient und leserlich zu speichern. Ohne Bitfelder wäre das nur sehr umständlich möglich. Das folgende Beispiel soll dies verdeutlichen:

In [ ]:
/* Windrichtung in Städten */
#include "stdio.h"

#define NORDEN 0
#define OSTEN 1
#define SUEDEN 2
#define WESTEN 3

/* Wir brauchen jetzt nur 2 Bit pro Stadt,
    weil es nur 4 Möglichkeiten gibt */
typedef struct Cities_s {
    char wien : 2;
    char graz : 2;
    char linz : 2;
    char salzburg : 2;
    char innsbruck : 2;
} Cities_t;

int main() {
    Cities_t wind = {0}; /* Variablen immer initialisieren */
    
    /* Die Speichergröße sind jetzt 2 Byte,
        weil 5 * 2 Bit = 10 Bit benötigen */
    printf("Speichergroesse: %ld Byte\n", sizeof(Cities_t));
    
    wind.wien = NORDEN;
    wind.salzburg = OSTEN;
    
    printf("Windrichtung in Salzburg: %d\n", wind.salzburg);
    printf("Windrichtung in Innsbruck: %d\n", wind.innsbruck);
    
    return 0;
}

### Erstellung einer eigenen Gleitkommazahl

Bitfelder können auch verwendet werden um eine Gleitkommazahl darzustellen. Ein `float` ist üblicherweise wie folgt aufgebaut:

<img src="images/float_bits.png" width=600>

Der Wert `x` der Gleitkommazahl wird dann folgendermaßen berechnet:

\begin{equation}
x = (-1)^{Vorzeichen} \quad \left(1 + \frac{Mantisse}{2^{23}}\right) \quad 2 ^ {Exponent-127}
\end{equation}

In [ ]:
/* Knovertierung von float zu seinen Bestandteilen */
#include "stdio.h"
#include "math.h"

typedef struct FloatingPoint_s {
    unsigned int mant : 23; /* Mantisse */
    unsigned int exp : 8; /* Exponent */
    unsigned int sign : 1; /* Vorzeichen */
} FloatingPoint_t;

int main() {
    FloatingPoint_t myFloat = {0};

    /* FloatingPoint_t ist 4 Byte groß,
        genau wie float */
    printf("FloatingPoint_t: %ld Byte\n", sizeof(FloatingPoint_t));
    printf("float:           %ld Byte\n", sizeof(float));
    
    {
        float pi = 3.141593;

        /* float kann zu FloatingPoint_t konvertiert werden */
        /* Zeiger aus dem float machen */
        float *pointer = &pi;
        /* Zu FloatingPoint_t Zeiger konvertieren */
        FloatingPoint_t* fPointer = (FloatingPoint_t*)pointer;
        /* Derefernzieren zu FloatingPoint_t,
            myFloat enthält nun genau die gleiche Serie von 0 und 1 wie pi */
        myFloat = *fPointer;
    
        printf("Gleitpunktzahl pi: %.7f\n", pi);
    
        /* Nun können die einzelnen Werte als Ganzzahlen ausgegeben werden */
        printf("\nDezimal:\n");
        printf("Mantisse:   %d\n", myFloat.mant);
        printf("Exponent:   %d\n", myFloat.exp);
        printf("Vorzeichen: %d\n", myFloat.sign);        
    }
    /* Berechnung des float Wertes */
    {
        float wert = pow(-1, myFloat.sign) * (1 + (float)myFloat.mant / (float)(1<<23)) * pow(2., myFloat.exp - 127);
        printf("Gleitpunktzahl Wert: %.7f\n", wert);
    }
    
    return 0;
}

## Anonyme Bitfelder

Bitfelder müssen keinen Namen haben, sondern können auch nur verwendet werden um einen ungenutzten "Zwischenraum" zu schaffen:

In [ ]:
#include "stdio.h"

typedef struct Bitfeld_s {
    char a : 6;
    char : 5; /* 5 Bit sind dazwischen leer, daher werden 3 Byte benötigt */
    char b : 6;
} Bitfeld_t;

int main() {
    printf("Speichergroesse: %ld Byte\n", sizeof(Bitfeld_t));
    
    return 0;
}

Anonyme Bitfelder mit der Länge `0` werden benutzt um das nächste Bitfeld erst in das nächste Maschinenwort zu schreiben. Das heißt, wenn `unsigned int` verwendet wurde, werden so viele Bit hinzugefügt, bis ein vielfaches von 4 Byte verbraucht wurde.

In [ ]:
#include "stdio.h"

/* Struktur ohne anonymen Bitfeld */
typedef struct Ohne_s { 
    unsigned int x : 5; 
    unsigned int y : 8; 
} Ohne_t;
  
/* Struktur mit anonymen Bitfeld der Länge 0.
    int benötigt 4 Byte, also hat diese Struktur nun 8 Byte */
typedef struct Mit_s { 
    unsigned int x : 5;
    unsigned int : 0; /* y kann nun erst nach 4 Byte beginnen */
    unsigned int y : 8; 
} Mit_t;

int main() {
    printf("Speichergroesse ohne: %ld Byte\n", sizeof(Ohne_t));
    printf("Speichergroesse mit:  %ld Byte\n", sizeof(Mit_t));
    
    return 0;
}

## Aufzählungen (*enumerations*)

Aufzählungen werden verwendet um mehrere ganzzahlige konstante Werte zu einem Typ zusammenzufassen. Die Definition sollte wie bei `struct` gleich mit einem `typedef` versehen werden damit dass Schlüsselwort `enum` nicht davor geschrieben werden muss.

So kann zum Beispiel einfach ein Wahrheitswert erstellt werden:

In [ ]:
#include "stdio.h"

typedef enum Boolean_e {
    FALSE = 0,
    TRUE = 1
} Boolean_t;

int main() {
    Boolean_t wahrheitswert = TRUE;
    
    /* Jetzt kann wahrheitswert direkt verwendet werden */
    if(wahrheitswert) {
        printf("Es ist wahr!\n");
    }
    
    printf("%d\n", wahrheitswert);
    
    return 0;
}

**ACHTUNG**: Die Werte von `FALSE` und `TRUE` müssen hier nicht explizit gesetzt werden, obwohl das auf jeden Fall empfohlen ist um die Leserlichkeit des Programmes zu erhöhen. Alternativ könnte in der `if` Bedingung auch expliziter sein, also `if(wahrheitswert == TRUE)`.

In [ ]:
#include "stdio.h"

/* Diese Schreibweise sollte wenn möglich vermieden werden!! */
typedef enum Boolean_e {
    FALSE,
    TRUE
} Boolean_t;

int main() {
    Boolean_t wahrheitswert = TRUE;
    
    /* Funktioniert zwar auch, macht es aber schwerer für einen Programmierer zu verstehen */
    if(wahrheitswert) {
        printf("Es ist wahr!\n");
    }
    
    printf("%d\n", wahrheitswert);
    
    return 0;
}

### Beispiel für Aufzählungen

Aufzählungen könnten verwendet werden um das obige Beispiel der Windrichtungen leserlicher zu gestalten:

In [ ]:
/* Windrichtung in Städten */
#include "stdio.h"

typedef enum Richtungen_e {
    NORDEN = 0,
    OSTEN = 1,
    SUEDEN = 2,
    WESTEN = 3
} Richtungen_t;

/* Eine switch Anweisung ist wie gemacht um
    eine Aufzählung zu überprüfen */
void printRichtung(Richtungen_t dir) {
    switch(dir) {
        case NORDEN: printf("Norden");
                     break;
        case OSTEN:  printf("Osten");
                     break;
        case SUEDEN: printf("Sueden");
                     break;
        case WESTEN: printf("Westen");
                     break;
        default: printf("ERROR: Richtung %d nicht bekannt!\n", dir);
    }
}

/* Wir brauchen nur 2 Bit pro Stadt,
    weil es nur 4 Möglichkeiten gibt */
typedef struct Cities_s {
    char wien : 2;
    char graz : 2;
    char linz : 2;
    char salzburg : 2;
    char innsbruck : 2;
} Cities_t;

int main() {
    Cities_t wind = {0}; /* Gesamte Struktur mit 0 initialisieren */
    
    wind.wien = NORDEN;
    wind.salzburg = OSTEN;
    
    /* Die Ausgabe einer Aufzählung liefert den zugewiesenen Wert */
    printf("Windrichtung in Salzburg: %d, also ", wind.salzburg);
    /* Mit einer Helferfunktion kann die tatsächliche Richtung ausgegeben werden */
    printRichtung(wind.salzburg);
    printf("\n");
    
    return 0;
}